In [5]:
import os

root = "/Users/bourymbodj/Projects/nmp_qc/nmp_qc/data//qm9/dsgdb9nsd/"
file_list = os.listdir(root)

num_mols = len(file_list)
print(num_mols)

133885


In [6]:

def read_xyz(file_name):
    with open(file_name, 'rb') as file:
        num_atoms= (int)(file.readline()) 
       # n=";".join(num_atoms)
        #num_atoms=(n.split(";"))
        properties = file.readline().split()[5:17] # only take the properties used in the experiments 
        properties = [num.replace(b'*^', b'e') for num in properties] 
        properties = [float(prop) for prop in properties]
        atom_types = [0]*num_atoms
        coords = np.array(np.zeros([num_atoms,3]))
        for na in range(num_atoms):
            coord_line = file.readline().split()
            atom_types[na] = coord_line[0]
            xyz_coords = coord_line[1:4]
            xyz_coords = [num.replace(b'*^', b'e') for num in xyz_coords] 
            coords[na,:] = 0#[float(num) for num in xyz_coords]  
        vib_freqs = file.readline()
        smiles = file.readline().split()[0]
        inchis = file.readline()
        
    return smiles, properties, atom_types, coords

In [7]:
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [8]:
import multiprocessing as mp
import numpy as np
from multiprocessing.pool import ThreadPool
import time
num=0

start = time.time()
N= mp.cpu_count()
files = os.scandir(root)
print (N)
with mp.pool.ThreadPool(processes = 8) as p:
        results= p.map(read_xyz, [root+file.name for file in files])
      
end = time.time()
print(end - start)

4
63.70721101760864


In [9]:
def column(matrix, i):
    return [row[i] for row in matrix]

smiles =column(results,0)
properties =column(results,1)

In [10]:
print (smiles)

[b'N#CC1NC11C2CC1C2', b'OC1=NON=C1OC=O', b'CC12CCC1CN1CC21', b'CN=C1OC(=O)CC1N', b'CC(CO)C(CO)C#N', b'CC1(C)CC(=O)OC1=N', b'CC1NC2(CNC12)C#N', b'OCCOC(=O)N1CC1', b'O=CC1=CN2CCC2=N1', b'NC1(COC1=N)C1CN1', b'CCN1C=NC(=C1)C#C', b'O=C1CC2COCC1O2', b'CC(=O)CC1(C)COC1', b'O=CC1=NCCO1', b'N#CC1(CC2CC2)CO1', b'CC1=CCCNC1C#N', b'O=C1NC2CCC3C2N13', b'N#CC12OC3COC1C23', b'CNC1=C(N)N=NN1C', b'OC1(CC=O)CC=CC1', b'NC1=NC2CC2NC1=O', b'CN1C(=N)ON=C1C=O', b'OC1=CC=C(F)C=C1', b'CC(CO)N1N=NN=N1', b'C1C2NC2C11CCCO1', b'CC1(OCCO1)C1CO1', b'CC#CCC1(C)CN1', b'O=C1C2CC3CC1C23', b'C1OC2=NC3C4OC13C24', b'O=C1NCC11C2CC1O2', b'CN=C1N=NOC=C1F', b'CC12OC3C4C3C1(C)C24', b'CC1=NC=C(C=C1)C#N', b'C1CC11CC2N=COC12', b'C1OC23CC(C=C2)C13', b'O=CNC(C#N)C1CC1', b'CC1(CC(O)C#C)CC1', b'CCCCCN=COC', b'CC1C2C3CC2(C#N)C13', b'O=C1OC23CC(C2)C13', b'CC1CCC(C=O)=C1N', b'CC1=NC(=CC=C1)C#N', b'COC1=C(OC=N1)C#N', b'OC1C(COC1=O)C#N', b'CC(C#C)C1COCO1', b'OC1C2C(C1O)N2C=O', b'CCCOC1C2CCC12', b'CC(C1CCC1)C(C)=O', b'OC12C3CC1C(=O)C2C3', b

In [11]:
import pandas as pd

In [31]:
df1 = pd.DataFrame(properties)

In [32]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,5.0509,78.30,-0.2676,0.0141,0.2818,1120.8417,0.137571,-380.797809,-380.790510,-380.789566,-380.829714,28.666
1,2.5256,55.40,-0.2868,-0.0597,0.2271,1180.5099,0.064339,-525.783370,-525.775913,-525.774969,-525.816245,25.955
2,1.5279,83.85,-0.2229,0.0807,0.3036,1029.1179,0.196525,-367.186557,-367.178749,-367.177805,-367.218256,32.345
3,2.1100,72.11,-0.2575,-0.0123,0.2451,1230.8475,0.136089,-455.191771,-455.182799,-455.181854,-455.226039,32.000
4,2.7860,75.68,-0.2791,0.0257,0.3048,1496.1036,0.168867,-440.291337,-440.280566,-440.279622,-440.327569,37.969


In [33]:
s= [smile.decode() for smile in smiles]

In [34]:
df = pd.DataFrame(s)

In [35]:
df.head()

,0
0,N#CC1NC11C2CC1C2
1,OC1=NON=C1OC=O
2,CC12CCC1CN1CC21
3,CN=C1OC(=O)CC1N
4,CC(CO)C(CO)C#N


In [42]:
result=pd.concat([df,df1],axis=1)
result.head()

,0,0,1,2,3,4,5,6,7,8,9,10,11
0,N#CC1NC11C2CC1C2,5.0509,78.30,-0.2676,0.0141,0.2818,1120.8417,0.137571,-380.797809,-380.790510,-380.789566,-380.829714,28.666
1,OC1=NON=C1OC=O,2.5256,55.40,-0.2868,-0.0597,0.2271,1180.5099,0.064339,-525.783370,-525.775913,-525.774969,-525.816245,25.955
2,CC12CCC1CN1CC21,1.5279,83.85,-0.2229,0.0807,0.3036,1029.1179,0.196525,-367.186557,-367.178749,-367.177805,-367.218256,32.345
3,CN=C1OC(=O)CC1N,2.1100,72.11,-0.2575,-0.0123,0.2451,1230.8475,0.136089,-455.191771,-455.182799,-455.181854,-455.226039,32.000
4,CC(CO)C(CO)C#N,2.7860,75.68,-0.2791,0.0257,0.3048,1496.1036,0.168867,-440.291337,-440.280566,-440.279622,-440.327569,37.969


In [43]:
result.to_csv('QM9_Smiles_.csv', index=False)